In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#
try:
  del pd.DataFrame.missing
except AttributeError:
  pass
    
@pd.api.extensions.register_dataframe_accessor("missing")
class MissingMethods:
    def __init__(self,pandas_obj):
      self._obj = pandas_obj

    def number_missing(self) -> int:
      return self._obj.isna().sum().sum()

    def number_complete(self) -> int:
      return self._obj.size - self._obj.missing.number_missing()

# CONTEO DE VALORES MISSING POR VARIABLE(COLUMNAS)
    def missing_variable_summary(self) -> pd.DataFrame:
        return self._obj.isna().pipe(
            lambda df_1: (
                df_1.sum()
                .reset_index(name="n_missing")
                .rename(columns={"index":"variable"})
                .assign(
                    n_cases=len(df_1),
                    pct_missing=lambda df_2: df_2.n_missing / df_2.n_cases * 100,
                )
            )
        )

# CONTEO DE VALORES MISSING POR VARIABLE RESUMEN(COLUMNAS)
    def missing_variable_table(self) -> pd.DataFrame:
        return (
            self._obj.missing.missing_variable_summary()
            .value_counts("n_missing")
            .reset_index(name="n_variables")
            .rename(columns={"n_missing": "n_missing_in_variable"})
            .assign(
                pct_variables=lambda df: df.n_variables / df.n_variables.sum() * 100
            )
            .sort_values("pct_variables", ascending=False)
        )

# CONTEO DE VALORES MISSING POR CASOS(FILAS)
    def missing_case_summary(self) -> pd.DataFrame:
        return self._obj.assign(
            case=lambda df: df.index,
            n_missing=lambda df: df.apply(
                axis="columns", func=lambda row: row.isna().sum()
            ),
            pct_missing=lambda df: df["n_missing"] / df.shape[1] * 100,
        )[["case", "n_missing", "pct_missing"]]

# CONTEO DE VALORES MISSING POR CASOS RESUMEN(FILAS)
    def missing_case_table(self) -> pd.DataFrame():
        return (
            self._obj.missing.missing_case_summary()
            .value_counts("n_missing")
            .reset_index(name="n_cases")
            .rename(columns={"n_missing": "n_missing_in_case", 0: "n_cases"})
            .assign(pct_case=lambda df: df.n_cases / df.n_cases.sum() * 100)
            .sort_values("pct_case", ascending=False)
        )

    def sort_variables_by_missingness(self, ascending = False):

        return (
            self._obj
            .pipe(
                lambda df: (
                    df[df.isna().sum().sort_values(ascending = ascending).index]
                )
            )
        )

    def missing_variable_plot(self):
        df = self._obj.missing.missing_variable_summary().sort_values("n_missing")
        plot_range = range(1, len(df.index) + 1)
        plt.hlines(y=plot_range, xmin=0, xmax=df.n_missing, color="green")
        plt.plot(df.n_missing, plot_range, "o", color="green")
        plt.yticks(plot_range, df.variable)
        plt.grid(axis="y")
        plt.xlabel("Number missing")
        plt.ylabel("Variable")

    def missing_case_plot(self):
        df = self._obj.missing.missing_case_summary()
        sns.displot(data=df, x="n_missing", binwidth=1, color="blue")
        plt.grid(axis="x")
        plt.xlabel("Number of missings in case")
        plt.ylabel("Number of cases")
        

def imputar_recursivo(df, columnas=None):
    # Si no se especifica la lista de columnas, se usan todas
    if columnas is None:
        columnas = list(df.columns)
    # Caso base: si ya no quedan columnas, se retorna el DataFrame
    if not columnas:
        return df
    else:
        # Se toma la primera columna de la lista
        col = columnas.pop(0)
        # Verificamos si la columna es de tipo numérico y si tiene valores nulos
        if pd.api.types.is_numeric_dtype(df[col]) and df[col].isnull().sum() > 0:
            promedio = df[col].mean()
            df[col] = df[col].fillna(promedio)
            print(f"Imputando en columna '{col}' con promedio {promedio}")
        # Llamada recursiva con el resto de las columnas
        return imputar_recursivo(df, columnas)